# Interactive Display Examples for QS Research

This notebook demonstrates various ways to display interactive output in new display windows for quantitative research and trading strategy analysis.

## 1. Setup and Libraries

Import the necessary libraries for interactive displays and data analysis.

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import pytimetk as tk

# Interactive plotting
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

# Traditional plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter display utilities
from IPython.display import display, HTML, IFrame
import webbrowser
import subprocess
import os

# QSResearch (if available)
try:
    from qsresearch.utils.zipline import get_zipline_history
    from qsresearch.portfolio_analysis.returns import create_full_returns_tearsheet_from_zipline
    QSRESEARCH_AVAILABLE = True
except ImportError:
    QSRESEARCH_AVAILABLE = False
    print("QSResearch not available - using sample data")

print("Libraries imported successfully!")

## 2. Create Sample Performance Data

Generate sample trading performance data to demonstrate the visualization techniques.

In [ ]:
# Create sample performance data
np.random.seed(42)
dates = pd.date_range('2023-01-01', '2025-07-01', freq='D')
n_days = len(dates)

# Simulate portfolio returns
daily_returns = np.random.normal(0.0008, 0.02, n_days)  # ~20% annual return, 20% volatility
portfolio_value = 1000000 * np.cumprod(1 + daily_returns)

# Simulate benchmark (SPY) returns
benchmark_returns = np.random.normal(0.0005, 0.015, n_days)  # ~12% annual return, 15% volatility
benchmark_value = 1000000 * np.cumprod(1 + benchmark_returns)

# Create performance DataFrame
performance_df = pd.DataFrame({
    'date': dates,
    'portfolio_value': portfolio_value,
    'benchmark_value': benchmark_value,
    'portfolio_returns': daily_returns,
    'benchmark_returns': benchmark_returns,
    'excess_returns': daily_returns - benchmark_returns
})

performance_df.set_index('date', inplace=True)

# Add some performance metrics
performance_df['drawdown'] = (performance_df['portfolio_value'] / performance_df['portfolio_value'].cummax()) - 1
performance_df['rolling_sharpe'] = performance_df['portfolio_returns'].rolling(252).mean() / performance_df['portfolio_returns'].rolling(252).std() * np.sqrt(252)

print(f"Generated {len(performance_df)} days of performance data")
performance_df.head()

## 3. Interactive Plots with Plotly (New Browser Windows)

Plotly can automatically open plots in new browser windows, which is perfect for detailed analysis.

In [ ]:
# Configure Plotly to open in browser
pio.renderers.default = "browser"  # This will open plots in new browser windows

# Alternative: Use 'notebook' to display inline
# pio.renderers.default = "notebook"

print("Plotly configured to open in browser windows")

In [ ]:
# Create comprehensive portfolio performance dashboard
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=('Portfolio vs Benchmark Value', 'Daily Returns Distribution', 
                   'Drawdown', 'Rolling Sharpe Ratio', 
                   'Excess Returns', 'Cumulative Excess Returns'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]],
    vertical_spacing=0.08
)

# Portfolio vs Benchmark
fig.add_trace(
    go.Scatter(x=performance_df.index, y=performance_df['portfolio_value'],
               mode='lines', name='Portfolio', line=dict(color='blue')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=performance_df.index, y=performance_df['benchmark_value'],
               mode='lines', name='Benchmark (SPY)', line=dict(color='red')),
    row=1, col=1
)

# Returns distribution
fig.add_trace(
    go.Histogram(x=performance_df['portfolio_returns'], name='Portfolio Returns',
                 opacity=0.7, nbinsx=50),
    row=1, col=2
)

# Drawdown
fig.add_trace(
    go.Scatter(x=performance_df.index, y=performance_df['drawdown']*100,
               mode='lines', name='Drawdown %', fill='tonexty',
               line=dict(color='red')),
    row=2, col=1
)

# Rolling Sharpe
fig.add_trace(
    go.Scatter(x=performance_df.index, y=performance_df['rolling_sharpe'],
               mode='lines', name='Rolling Sharpe (252d)',
               line=dict(color='green')),
    row=2, col=2
)

# Excess returns
fig.add_trace(
    go.Scatter(x=performance_df.index, y=performance_df['excess_returns']*100,
               mode='lines', name='Daily Excess Returns %',
               line=dict(color='purple')),
    row=3, col=1
)

# Cumulative excess returns
cumulative_excess = (1 + performance_df['excess_returns']).cumprod() - 1
fig.add_trace(
    go.Scatter(x=performance_df.index, y=cumulative_excess*100,
               mode='lines', name='Cumulative Excess Returns %',
               line=dict(color='orange')),
    row=3, col=2
)

# Update layout
fig.update_layout(
    height=900,
    title_text="Portfolio Performance Dashboard",
    showlegend=True
)

# This will open in a new browser window
fig.show()

print("Portfolio performance dashboard opened in new browser window!")

In [ ]:
# Create risk metrics visualization
fig = go.Figure()

# Calculate rolling volatility
rolling_vol = performance_df['portfolio_returns'].rolling(63).std() * np.sqrt(252) * 100

fig.add_trace(go.Scatter(
    x=performance_df.index,
    y=rolling_vol,
    mode='lines',
    name='Rolling Volatility (63d)',
    line=dict(color='red', width=2)
))

# Add volatility bands
fig.add_hline(y=15, line_dash="dash", line_color="green", 
              annotation_text="Low Vol (15%)")
fig.add_hline(y=25, line_dash="dash", line_color="orange", 
              annotation_text="High Vol (25%)")

fig.update_layout(
    title="Portfolio Risk Metrics - Rolling Volatility",
    xaxis_title="Date",
    yaxis_title="Annualized Volatility (%)",
    height=500
)

# Open in new browser window
fig.show()

print("Risk metrics chart opened in new browser window!")

## 4. Matplotlib Figures in New Windows

For traditional matplotlib plots, we can configure them to open in separate windows.

In [ ]:
# Configure matplotlib for interactive display
%matplotlib qt  # Use Qt backend for separate windows
# Alternative backends: %matplotlib tk, %matplotlib gtk

plt.ion()  # Turn on interactive mode

print("Matplotlib configured for separate windows")

In [ ]:
# Create matplotlib figures in separate windows

# Figure 1: Performance comparison
fig1, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
fig1.suptitle('Portfolio Performance Analysis', fontsize=16)

# Portfolio value over time
ax1.plot(performance_df.index, performance_df['portfolio_value'], 
         label='Portfolio', linewidth=2, color='blue')
ax1.plot(performance_df.index, performance_df['benchmark_value'], 
         label='Benchmark', linewidth=2, color='red')
ax1.set_title('Portfolio vs Benchmark Value')
ax1.set_ylabel('Value ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Drawdown
ax2.fill_between(performance_df.index, performance_df['drawdown']*100, 0, 
                 alpha=0.3, color='red', label='Drawdown')
ax2.plot(performance_df.index, performance_df['drawdown']*100, 
         color='red', linewidth=1)
ax2.set_title('Portfolio Drawdown')
ax2.set_ylabel('Drawdown (%)')
ax2.set_xlabel('Date')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()  # Opens in new window

print("Matplotlib performance chart opened in new window!")

In [ ]:
# Figure 2: Returns analysis
fig2, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
fig2.suptitle('Returns Analysis Dashboard', fontsize=16)

# Returns distribution
ax1.hist(performance_df['portfolio_returns']*100, bins=50, alpha=0.7, 
         color='blue', label='Portfolio')
ax1.hist(performance_df['benchmark_returns']*100, bins=50, alpha=0.7, 
         color='red', label='Benchmark')
ax1.set_title('Daily Returns Distribution')
ax1.set_xlabel('Daily Returns (%)')
ax1.set_ylabel('Frequency')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Rolling correlation
rolling_corr = performance_df['portfolio_returns'].rolling(63).corr(
    performance_df['benchmark_returns'])
ax2.plot(performance_df.index, rolling_corr, color='green', linewidth=2)
ax2.set_title('Rolling Correlation (63 days)')
ax2.set_ylabel('Correlation')
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0.8, color='red', linestyle='--', alpha=0.7)

# Scatter plot of returns
ax3.scatter(performance_df['benchmark_returns']*100, 
           performance_df['portfolio_returns']*100, 
           alpha=0.5, s=1)
ax3.set_title('Portfolio vs Benchmark Returns')
ax3.set_xlabel('Benchmark Returns (%)')
ax3.set_ylabel('Portfolio Returns (%)')
ax3.grid(True, alpha=0.3)

# Add regression line
z = np.polyfit(performance_df['benchmark_returns']*100, 
               performance_df['portfolio_returns']*100, 1)
p = np.poly1d(z)
ax3.plot(performance_df['benchmark_returns']*100, 
         p(performance_df['benchmark_returns']*100), "r--", alpha=0.8)

# Monthly returns heatmap
monthly_returns = performance_df['portfolio_returns'].resample('M').apply(
    lambda x: (1 + x).prod() - 1) * 100
monthly_table = monthly_returns.groupby([monthly_returns.index.year, 
                                        monthly_returns.index.month]).mean().unstack()

im = ax4.imshow(monthly_table.values, cmap='RdYlGn', aspect='auto')
ax4.set_title('Monthly Returns Heatmap (%)')
ax4.set_xticks(range(12))
ax4.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
ax4.set_yticks(range(len(monthly_table.index)))
ax4.set_yticklabels(monthly_table.index)
ax4.set_ylabel('Year')

plt.tight_layout()
plt.show()  # Opens in new window

print("Returns analysis dashboard opened in new window!")

## 5. Opening Web Interfaces in New Windows

For tools like MLflow, Jupyter Lab, or other web-based interfaces.

In [ ]:
# Function to start MLflow server and open in browser
def open_mlflow_ui(port=5000):
    """Start MLflow server and open UI in browser"""
    import subprocess
    import webbrowser
    import time
    
    try:
        # Start MLflow server in background
        process = subprocess.Popen(
            ["mlflow", "ui", "--host", "0.0.0.0", "--port", str(port)],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        
        # Wait a moment for server to start
        time.sleep(3)
        
        # Open in browser
        url = f"http://localhost:{port}"
        webbrowser.open_new_tab(url)
        
        print(f"MLflow UI opened at {url}")
        print(f"Server process ID: {process.pid}")
        
        return process
        
    except Exception as e:
        print(f"Error starting MLflow: {e}")
        return None

# Uncomment to start MLflow UI
# mlflow_process = open_mlflow_ui()

print("MLflow UI function ready - uncomment to use")

In [ ]:
# Function to open various external tools
def open_external_tool(tool_type="jupyter", port=8888):
    """Open external tools in browser"""
    
    if tool_type == "jupyter":
        url = f"http://localhost:{port}"
        webbrowser.open_new_tab(url)
        print(f"Opening Jupyter at {url}")
        
    elif tool_type == "tensorboard":
        url = f"http://localhost:{port}"
        webbrowser.open_new_tab(url)
        print(f"Opening TensorBoard at {url}")
        
    elif tool_type == "streamlit":
        url = f"http://localhost:{port}"
        webbrowser.open_new_tab(url)
        print(f"Opening Streamlit app at {url}")

# Example usage:
# open_external_tool("jupyter", 8888)
# open_external_tool("mlflow", 5000)

print("External tool functions ready")

## 6. Interactive Jupyter Widgets

Create interactive widgets for parameter exploration that can be displayed in separate output areas.

In [ ]:
# Install ipywidgets if not available
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    WIDGETS_AVAILABLE = True
except ImportError:
    print("Installing ipywidgets...")
    import subprocess
    subprocess.check_call(["pip", "install", "ipywidgets"])
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    WIDGETS_AVAILABLE = True

print("Widgets ready!")

In [ ]:
# Create interactive parameter dashboard
if WIDGETS_AVAILABLE:
    # Create widgets
    lookback_slider = widgets.IntSlider(
        value=252,
        min=30,
        max=500,
        step=10,
        description='Lookback:'
    )
    
    metric_dropdown = widgets.Dropdown(
        options=['portfolio_value', 'drawdown', 'rolling_sharpe'],
        value='portfolio_value',
        description='Metric:'
    )
    
    plot_button = widgets.Button(
        description='Update Plot',
        button_style='success'
    )
    
    output_area = widgets.Output()
    
    def update_plot(b):
        with output_area:
            clear_output(wait=True)
            
            # Create new plot based on parameters
            fig = go.Figure()
            
            if metric_dropdown.value == 'portfolio_value':
                y_data = performance_df['portfolio_value'].tail(lookback_slider.value)
                title = f"Portfolio Value (Last {lookback_slider.value} days)"
            elif metric_dropdown.value == 'drawdown':
                y_data = performance_df['drawdown'].tail(lookback_slider.value) * 100
                title = f"Drawdown % (Last {lookback_slider.value} days)"
            else:
                y_data = performance_df['rolling_sharpe'].tail(lookback_slider.value)
                title = f"Rolling Sharpe (Last {lookback_slider.value} days)"
            
            x_data = performance_df.index.tail(lookback_slider.value)
            
            fig.add_trace(go.Scatter(
                x=x_data,
                y=y_data,
                mode='lines',
                name=metric_dropdown.value
            ))
            
            fig.update_layout(
                title=title,
                height=400
            )
            
            # Display inline (or use fig.show() for new window)
            fig.show()
    
    plot_button.on_click(update_plot)
    
    # Display dashboard
    dashboard = widgets.VBox([
        widgets.HBox([lookback_slider, metric_dropdown, plot_button]),
        output_area
    ])
    
    display(dashboard)
    
    print("Interactive dashboard created! Use the controls above to explore data.")
else:
    print("Widgets not available")

## 7. Export and Open HTML Reports

Create standalone HTML reports that open in new browser windows.

In [ ]:
# Create comprehensive HTML report
def create_html_report(performance_df, filename="portfolio_report.html"):
    """Create a comprehensive HTML report"""
    
    # Calculate key metrics
    total_return = (performance_df['portfolio_value'].iloc[-1] / performance_df['portfolio_value'].iloc[0] - 1) * 100
    annual_return = ((performance_df['portfolio_value'].iloc[-1] / performance_df['portfolio_value'].iloc[0]) ** (252 / len(performance_df)) - 1) * 100
    volatility = performance_df['portfolio_returns'].std() * np.sqrt(252) * 100
    sharpe_ratio = performance_df['portfolio_returns'].mean() / performance_df['portfolio_returns'].std() * np.sqrt(252)
    max_drawdown = performance_df['drawdown'].min() * 100
    
    # Create main performance chart
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(
        x=performance_df.index,
        y=performance_df['portfolio_value'],
        mode='lines',
        name='Portfolio Value',
        line=dict(color='blue', width=2)
    ))
    fig1.update_layout(
        title="Portfolio Performance Over Time",
        xaxis_title="Date",
        yaxis_title="Portfolio Value ($)",
        height=500
    )
    
    # Create drawdown chart
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(
        x=performance_df.index,
        y=performance_df['drawdown'] * 100,
        mode='lines',
        fill='tonexty',
        name='Drawdown %',
        line=dict(color='red')
    ))
    fig2.update_layout(
        title="Portfolio Drawdown",
        xaxis_title="Date",
        yaxis_title="Drawdown (%)",
        height=400
    )
    
    # Convert plots to HTML
    plot1_html = fig1.to_html(include_plotlyjs='cdn', div_id="plot1")
    plot2_html = fig2.to_html(include_plotlyjs='cdn', div_id="plot2")
    
    # Create HTML template
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Portfolio Performance Report</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 40px; }}
            .metrics {{ display: grid; grid-template-columns: repeat(3, 1fr); gap: 20px; margin: 20px 0; }}
            .metric-card {{ background: #f5f5f5; padding: 20px; border-radius: 8px; text-align: center; }}
            .metric-value {{ font-size: 24px; font-weight: bold; color: #2E86AB; }}
            .metric-label {{ font-size: 14px; color: #666; }}
            h1 {{ color: #2E86AB; text-align: center; }}
            h2 {{ color: #555; border-bottom: 2px solid #2E86AB; padding-bottom: 10px; }}
        </style>
    </head>
    <body>
        <h1>Portfolio Performance Report</h1>
        <p><strong>Report Generated:</strong> {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
        
        <h2>Key Performance Metrics</h2>
        <div class="metrics">
            <div class="metric-card">
                <div class="metric-value">{total_return:.2f}%</div>
                <div class="metric-label">Total Return</div>
            </div>
            <div class="metric-card">
                <div class="metric-value">{annual_return:.2f}%</div>
                <div class="metric-label">Annualized Return</div>
            </div>
            <div class="metric-card">
                <div class="metric-value">{volatility:.2f}%</div>
                <div class="metric-label">Volatility</div>
            </div>
            <div class="metric-card">
                <div class="metric-value">{sharpe_ratio:.2f}</div>
                <div class="metric-label">Sharpe Ratio</div>
            </div>
            <div class="metric-card">
                <div class="metric-value">{max_drawdown:.2f}%</div>
                <div class="metric-label">Max Drawdown</div>
            </div>
            <div class="metric-card">
                <div class="metric-value">{len(performance_df)}</div>
                <div class="metric-label">Trading Days</div>
            </div>
        </div>
        
        <h2>Portfolio Performance</h2>
        {plot1_html}
        
        <h2>Risk Analysis</h2>
        {plot2_html}
        
        <h2>Summary</h2>
        <p>This portfolio generated a total return of <strong>{total_return:.2f}%</strong> 
        over {len(performance_df)} trading days, with an annualized return of <strong>{annual_return:.2f}%</strong> 
        and volatility of <strong>{volatility:.2f}%</strong>.</p>
        
        <p>The Sharpe ratio of <strong>{sharpe_ratio:.2f}</strong> indicates 
        {'excellent' if sharpe_ratio > 2 else 'good' if sharpe_ratio > 1 else 'moderate'} 
        risk-adjusted performance.</p>
        
        <p>Maximum drawdown was <strong>{max_drawdown:.2f}%</strong>, showing the largest peak-to-trough decline.</p>
    </body>
    </html>
    """
    
    # Save to file
    with open(filename, 'w') as f:
        f.write(html_content)
    
    # Open in browser
    webbrowser.open_new_tab(f'file://{os.path.abspath(filename)}')
    
    print(f"HTML report saved as {filename} and opened in browser!")
    return filename

# Create and open the report
report_file = create_html_report(performance_df)
print(f"Report available at: {os.path.abspath(report_file)}")

## Summary

This notebook demonstrated several ways to display interactive output in new display windows:

1. **Plotly with Browser Renderer**: Set `pio.renderers.default = "browser"` to automatically open plots in new browser windows

2. **Matplotlib with Qt Backend**: Use `%matplotlib qt` to open plots in separate desktop windows

3. **Web Interface Integration**: Functions to start and open MLflow UI, Jupyter Lab, and other web-based tools

4. **Interactive Jupyter Widgets**: Create parameter exploration dashboards within the notebook

5. **HTML Report Export**: Generate standalone HTML reports with embedded interactive charts

### For Your QS Research Workflow:

- Use **Plotly browser mode** for detailed performance analysis that you want to examine in full-screen
- Use **matplotlib Qt windows** for traditional statistical charts and quick analysis
- Use **MLflow UI** for comparing multiple strategy runs and experiments
- Use **HTML reports** for sharing results with stakeholders
- Use **Jupyter widgets** for interactive parameter tuning during strategy development

The key is choosing the right display method for your specific use case and audience!